# 黑色金属 - 焦炭

In [2]:
# 设置工作目录
import os
os.getcwd()
if os.name == 'posix':
    os.chdir('/Volumes/Repository/Projects/ffa/')
else:
    os.chdir("E:\\Document\\Project\\ffa")

In [3]:
# 加载依赖模块
import pandas as pd
import akshare as ak
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from datetime import datetime, date
import importlib
import commodity
import json
import dataworks as dw

In [45]:
# 重新加载salary计算模块
importlib.reload(commodity)

<module 'commodity' from '/Volumes/Repository/Projects/ffa/commodity.py'>

## 数据准备与预处理

In [4]:
symbol_id = 'J'
symbol_name = '焦炭'
fBasePath = 'steel/data/up-stream/焦炭/'
json_file = './steel/setting.json'

### 数据索引设置

In [5]:
# 品种数据索引初始化
# 首次使用json配置文件存取品种的数据索引
data_index = {#'主力合约': {'Name': "dominant_contract", 'Source':'AKShare', 'Path': fBasePath + '.xlsx', 'Field':'dominant_contract', 'DataFrame': "df_basis"},
                #'近月合约': {'Name': "near_contract", 'Source':'AKShare', 'Path': fBasePath + '.xlsx', 'Field':'near_contract', 'DataFrame': "df_basis"},
                '主力合约收盘价': {'Name': "dominant_close_price", 'Source':'SQLite', 'Path': 'dominant', 'Field':'收盘价', 'DataFrame': "df_dominant"},
                '主力合约结算价': {'Name': "dominant_settle_price", 'Source':'SQLite', 'Path': 'dominant', 'Field':'动态结算价', 'DataFrame': "df_dominant"},
                '主力合约持仓量': {'Name': "dominant_open_interest", 'Source':'SQLite', 'Path': 'dominant', 'Field':'持仓量', 'DataFrame': "df_dominant"},
                '近月合约收盘价': {'Name': "near_close_price", 'Source':'SQLite', 'Path': 'near', 'Field':'close', 'DataFrame': "df_near"},
                '近月合约结算价': {'Name': "near_settle_price", 'Source':'SQLite', 'Path': 'near', 'Field':'settle', 'DataFrame': "df_near"},
                '近月合约持仓量': {'Name': "near_open_interest", 'Source':'SQLite', 'Path': 'near', 'Field':'open_interest', 'DataFrame': "df_near"}, 
                '现货价格': {'Name': "spot_price", 'Source':'Choice', 'Path': fBasePath + '焦炭基差.xlsx', 'Field':'现货价格:焦炭', 'DataFrame': "df_basis"},
                #'基差': {'Name': "basis", 'Source':'AKShare', 'Path': fBasePath + '.xlsx', 'Field':'dom_basis', 'DataFrame': "df_basis"},
                #'基差率': {'Name': "basis_rate", 'Source':'AKShare', 'Path': fBasePath + '.xlsx', 'Field':'dom_basis_rate', 'DataFrame': "df_basis"},
                '产量': {'Name': "production", 'Source':'Choice', 'Path': fBasePath + '焦炭产量.xlsx', 'Field':'产量:焦炭:当月值', 'DataFrame': "df_prodoction"},
                '销量': {'Name': "sales", 'Source':'Choice', 'Path': fBasePath + '焦炭销量.xlsx', 'Field':'焦炭消费量:总计', 'DataFrame': "df_sales"},
                '库存': {'Name': "instock", 'Source':'Choice', 'Path': fBasePath + '焦炭库存.xlsx', 'Field':'焦炭库存:样本钢厂:全国', 'DataFrame': "df_instock"}, #需要求和
                '仓单': {'Name': "receipt", 'Source':'Choice', 'Path': fBasePath + '焦炭库存.xlsx', 'Field':'仓单数量:焦炭', 'DataFrame': "df_instock"}
                }
profit_formula = {'Name': 'profit_formula', 'Factor': {'X': 1.0, 'Y': 1.0}, '其他成本': 0}
dominant_months = [1, 5, 10]
exchange_id = 'dce'
symbol_setting = {'DataIndex': data_index,
                  'ProfitFormula': profit_formula,
                  'DominantMonths': dominant_months,
                  'ExchangeID': exchange_id}


### 数据预处理

In [6]:
# 构造品种数据访问对象
symbol = commodity.SymbolData(symbol_id, symbol_name, json_file, symbol_setting)
# symbol = commodity.SymbolData(symbol_id, symbol_name, json_file)
dws = dw.DataWorks()
merged_data = symbol.merge_data(dws)

### 更新数据

- 通过AKshare接口更新现货价格数据
- 手动更新Choice导出数据（使用Microsoft Excel打开，自动更新后保存）

In [ ]:
# 更新现货价格数据 - AKShare
# 追加最新数据
df_basis = symbol.update_akshare_file(mode='append')
# 2011、2012年数据待更新
# symbol_data.update_akshare_file(mode='period', start_date='20120101', end_date='20121231')

In [17]:
futures_spot_price_df = ak.futures_spot_price("20231130")

In [ ]:
df_append = ak.futures_spot_price_daily(start_day='20231101', end_day='20231130', vars_list=[symbol_id])